In [1]:
!pip install transformers -U -q
! pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 772.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 26.0 MB/s eta 0:00:00


In [2]:
!pip freeze | grep transformers

transformers==4.45.2


In [3]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import os

In [4]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [5]:
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")

tokenizer_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/717 [00:00<?, ?B/s]

In [6]:
language_codes = {
    "Hindi": "hi_IN",
    "Gujarati": "gu_IN",
    "Bengali": "bn_IN",
    "Tamil": "ta_IN",
    "Telugu": "te_IN"
}

In [7]:
def translate_text(text, target_lang_code):
    # Tokenize the input text
    model_inputs = tokenizer(text, return_tensors="pt")

    # Generate translation
    generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id[target_lang_code]
    )

    # Decode the translated tokens
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    return translation[0]

while True:
    # Option for file input
    file_input = input("Do you want to input text from a file? (yes/no): ").strip().lower()

    if file_input == 'yes':
        file_path = input("Enter the path to the text file (or 'exit' to quit): ")
        if file_path.lower() == 'exit':
            print("Exiting the translation tool.")
            break
        if not os.path.isfile(file_path):
            print("File not found. Please enter a valid file path.")
            continue
        with open(file_path, 'r') as file:
            text_to_translate = file.read()
            print(f"Input text from file: {text_to_translate}")

    elif file_input == 'no':
        text_to_translate = input("Enter the text in English (or 'exit' to quit): ")
        if text_to_translate.lower() == 'exit':
            print("Exiting the translation tool.")
            break

    else:
        print("Invalid input. Please type 'yes' or 'no'.")
        continue

    # Display available language options
    print("Available languages:")
    for i, lang in enumerate(language_codes.keys(), 1):
        print(f"{i}. {lang}")

    # Get user choice for target language
    lang_choice = input("Select the target language (by number): ")

    if lang_choice.lower() == 'exit':
        print("Exiting the translation tool.")
        break

    try:
        lang_choice = int(lang_choice)  # Convert to integer
        if lang_choice < 1 or lang_choice > len(language_codes):
            print("Invalid choice. Please select a valid language number.")
            continue
    except ValueError:
        print("Invalid input. Please enter a number corresponding to the language or 'exit' to quit.")
        continue

    # Get the language code from the user's selection
    selected_language = list(language_codes.keys())[lang_choice - 1]
    target_language_code = language_codes[selected_language]

    # Perform translation
    translated_text = translate_text(text_to_translate, target_language_code)
    print(f"Translated text in {selected_language}: {translated_text}\n")



Do you want to input text from a file? (yes/no): yes
Enter the path to the text file (or 'exit' to quit): C:\Users\dell\OneDrive\Desktop\test.txt
File not found. Please enter a valid file path.
Do you want to input text from a file? (yes/no): yes
Enter the path to the text file (or 'exit' to quit): "C:\Users\dell\OneDrive\Desktop\test.txt"
File not found. Please enter a valid file path.
Do you want to input text from a file? (yes/no): yes
Enter the path to the text file (or 'exit' to quit): /content/test.txt
Input text from file: how are you

Available languages:
1. Hindi
2. Gujarati
3. Bengali
4. Tamil
5. Telugu
Select the target language (by number): 3
Translated text in Bengali: আপনি কেমন আছেন

Do you want to input text from a file? (yes/no): no
Enter the text in English (or 'exit' to quit): exit
Exiting the translation tool.


Writing app.py
